In [ ]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

##### Read all the data as required

In [ ]:
from src.formula1.formula1_constants import *
from src.formula1.formula1_utils import *

In [ ]:
drivers_df = (
    spark.read.table("f1_silver.drivers")
    .withColumnRenamed("number", "driver_number")
    .withColumnRenamed("name", "driver_name")
    .withColumnRenamed("nationality", "driver_nationality")
)

In [ ]:
constructors_df = spark.read.table("f1_silver.constructors").withColumnRenamed(
    "name", "team"
)

In [ ]:
circuits_df = spark.read.table("f1_silver.circuits").withColumnRenamed(
    "location", "circuit_location"
)

In [ ]:
races_df = (
    spark.read.table("f1_silver.races")
    .withColumnRenamed("name", "race_name")
    .withColumnRenamed("race_timestamp", "race_date")
)

In [ ]:
results_df = (
    spark.read.table("f1_silver.results")
    .filter(f"file_date = '{v_file_date}'")
    .withColumnRenamed("time", "race_time")
    .withColumnRenamed("race_id", "result_race_id")
    .withColumnRenamed("file_date", "result_file_date")
)

##### Join circuits to races

In [ ]:
race_circuits_df = races_df.join(
    circuits_df, races_df.circuit_id == circuits_df.circuit_id, "inner"
).select(
    races_df.race_id,
    races_df.race_year,
    races_df.race_name,
    races_df.race_date,
    circuits_df.circuit_location,
)

##### Join results to all other dataframes

In [ ]:
race_results_df = (
    results_df.join(
        race_circuits_df, results_df.result_race_id == race_circuits_df.race_id
    )
    .join(drivers_df, results_df.driver_id == drivers_df.driver_id)
    .join(constructors_df, results_df.constructor_id == constructors_df.constructor_id)
)

In [ ]:
from pyspark.sql.functions import current_timestamp

In [ ]:
final_df = (
    race_results_df.select(
        "race_id",
        "race_year",
        "race_name",
        "race_date",
        "circuit_location",
        "driver_name",
        "driver_number",
        "driver_nationality",
        "team",
        "grid",
        "fastest_lap",
        "race_time",
        "points",
        "position",
        "result_file_date",
    )
    .withColumn("created_date", current_timestamp())
    .withColumnRenamed("result_file_date", "file_date")
)

In [ ]:
# merge_condition = "tgt.driver_name = src.driver_name AND tgt.race_id = src.race_id"
# merge_delta_data(final_df, 'f1_gold', 'race_results', presentation_folder_path, merge_condition, 'race_id')

final_df.write.mode("overwrite").partitionBy("race_id").format("delta").saveAsTable(
    "f1_gold.race_results"
)

In [ ]:
%sql
SELECT * FROM f1_gold.race_results;